In [1]:
from src import reaction_string_parser

Initialize an instance of reaction string parser
=====

In [2]:
rsp = reaction_string_parser.ReactionStringParser()
left, right, rate, direction = \
    rsp.parse_reaction_string("A + 5c + 0.8 ABUGH  +1/ 2 AUGH --> 5J , k21")
print((left, right, rate, direction))

('A + 5c + 0.8 ABUGH  +1/ 2 AUGH', '5J', 'k21', 1)


Using the all-in-one parser
====

In [3]:
# Example usage:
reaction_strings = ["A + B -> C, kon", "2X -> Y, kf", "C -> A + B, koff", "Y + A -> X + C, ki"]
species_names, rate_constant_names, reactant_matrix, product_matrix =\
        rsp.parse_reaction_strings(reaction_strings)
print("Species Names:")
print(species_names)
print("Reactant Stoichiometry Matrix:")
print(reactant_matrix)
print("Product Stoichiometry Matrix:")
print(product_matrix)
print("Rate Constant Values: Placeholder. for future use, I think rate constant not from string is better for now?")
print("Rate Constant Names:")
print(rate_constant_names)

Species Names:
['B', 'A', 'X', 'C', 'Y']
Reactant Stoichiometry Matrix:
[[1 1 0 0 0]
 [0 0 2 0 0]
 [0 0 0 1 0]
 [0 1 0 0 1]]
Product Stoichiometry Matrix:
[[0 0 0 1 0]
 [0 0 0 0 1]
 [1 1 0 0 0]
 [0 0 1 1 0]]
Rate Constant Values: Placeholder. for future use, I think rate constant not from string is better for now?
Rate Constant Names:
['kon', 'kf', 'koff', 'ki']


Sorting
===

In [4]:
# sorting while parsing
# Example usage:
reaction_strings = ["A + B -> C, kon", "2X -> Y, kf", "C -> A + B, koff", "Y + A -> X + C, ki"]
species_names, rate_constant_names, reactant_matrix, product_matrix =\
        rsp.parse_reaction_strings(reaction_strings, sort_species_by="increasing")
print("Species Names:")
print(species_names)
print("Reactant Stoichiometry Matrix:")
print(reactant_matrix)
print("Product Stoichiometry Matrix:")
print(product_matrix)
print("Rate Constant Values: Placeholder. for future use, I think rate constant not from string is better for now?")
print("Rate Constant Names:")
print(rate_constant_names)

Species Names:
['A' 'B' 'C' 'X' 'Y']
Reactant Stoichiometry Matrix:
[[1 1 0 0 0]
 [0 0 0 2 0]
 [0 0 1 0 0]
 [1 0 0 0 1]]
Product Stoichiometry Matrix:
[[0 0 1 0 0]
 [0 0 0 0 1]
 [1 1 0 0 0]
 [0 0 1 1 0]]
Rate Constant Values: Placeholder. for future use, I think rate constant not from string is better for now?
Rate Constant Names:
['kon', 'kf', 'koff', 'ki']


In [5]:
# sorting after parsing
# sort by rate constant
rsp.sort_by_rate_constants(reactant_matrix, product_matrix,
            rate_constant_names, sort_order = ['kon', 'koff', 'kf', 'ki'])

(array([[1, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 2, 0],
        [1, 0, 0, 0, 1]]),
 array([[0, 0, 1, 0, 0],
        [1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [0, 0, 1, 1, 0]]),
 array(['kon', 'koff', 'kf', 'ki'], dtype='<U4'))

In [6]:
# sort by species names
rsp.sort_by_species_names(reactant_matrix, product_matrix,
            species_names, sort_order = ['A', 'B', 'C', 'X', 'Y'])

(array([[1, 1, 0, 0, 0],
        [0, 0, 0, 2, 0],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 1]]),
 array([[0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1],
        [1, 1, 0, 0, 0],
        [0, 0, 1, 1, 0]]),
 array(['A', 'B', 'C', 'X', 'Y'], dtype='<U1'))

Decimal / Fraction support
======

In [7]:
# Example usage:
# Remember to set dtype to float to enable decimal / fraction support!
reaction_strings = ["0.6A + 1/4 B -> C, kon", "C -> 0.6 A + 1/4 B, koff"]
species_names, rate_constant_names, reactant_matrix, product_matrix =\
        rsp.parse_reaction_strings(reaction_strings, dtype = float)
print("Species Names:")
print(species_names)
print("Reactant Stoichiometry Matrix:")
print(reactant_matrix)
print("Product Stoichiometry Matrix:")
print(product_matrix)
print("Rate Constant Values: Placeholder. for future use, I think rate constant not from string is better for now?")
print("Rate Constant Names:")
print(rate_constant_names)

Species Names:
['B', 'A', 'C']
Reactant Stoichiometry Matrix:
[[0.25 0.6  0.  ]
 [0.   0.   1.  ]]
Product Stoichiometry Matrix:
[[0.   0.   1.  ]
 [0.25 0.6  0.  ]]
Rate Constant Values: Placeholder. for future use, I think rate constant not from string is better for now?
Rate Constant Names:
['kon', 'koff']


Functionality breakdown
===
Parse LHS or RHS of reaction string to dictionaries
===

In [8]:
rsp.parse_stoichiometry_string(left)

{'A': 1.0, 'c': 5.0, 'ABUGH': 0.8, 'AUGH': 0.5}

Extract Reactant and Product dict from strings
===

In [9]:
reaction_strings = ["A + B -> C, kon", "2X -> Y, koff", "C -> A + B, koff", 
                    "Y + A -> X + C, ki",
                    "A + 5c + 0.8 ABUGH  +1/ 2 AUGH --> 5J , k21"]
rsp.extract_species_dictionaries_from_reaction_strings(reaction_strings)

 This can lead to ambiguity in ODE definition. 
 Please consider change the naming.


({'A', 'ABUGH', 'AUGH', 'B', 'C', 'J', 'X', 'Y', 'c'},
 ['kon', 'koff', 'koff', 'ki', 'k21'],
 [{'A': 1.0, 'B': 1.0},
  {'X': 2.0},
  {'C': 1.0},
  {'Y': 1.0, 'A': 1.0},
  {'A': 1.0, 'c': 5.0, 'ABUGH': 0.8, 'AUGH': 0.5}],
 [{'C': 1.0},
  {'Y': 1.0},
  {'A': 1.0, 'B': 1.0},
  {'X': 1.0, 'C': 1.0},
  {'J': 5.0}])